In [1]:
import sys
from pathlib import Path

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))


In [2]:
import pandas as pd
from paths import PROCESSED_DIR

df = pd.read_parquet(PROCESSED_DIR / 'ts_df_bolleria_20250801.parquet')
df

,base_imponible,is_summer_peak,is_easter,week_start,base_imponible_lag1,base_imponible_lag2,base_imponible_lag3,base_imponible_lag4,base_imponible_lag12,base_imponible_lag24,base_imponible_lag52,base_imponible_next1
0,597.65,0,0,2024-01-15,572.51,534.79,563.18,806.54,912.13,1745.97,825.11,680.30
1,680.30,0,0,2024-01-22,597.65,572.51,534.79,563.18,750.99,1681.41,658.40,603.99
2,603.99,0,0,2024-01-29,680.30,597.65,572.51,534.79,821.84,1753.02,741.40,600.14
3,600.14,0,0,2024-02-05,603.99,680.30,597.65,572.51,607.02,1835.18,653.64,689.32
4,689.32,0,0,2024-02-12,600.14,603.99,680.30,597.65,304.73,2127.71,680.46,627.76
...,...,...,...,...,...,...,...,...,...,...,...,...
68,756.42,0,0,2025-05-19,810.97,842.62,891.38,894.20,725.72,756.58,828.30,802.16
69,802.16,0,0,2025-05-26,756.42,810.97,842.62,891.38,635.99,604.75,854.34,881.72
70,881.72,0,0,2025-06-02,802.16,756.42,810.97,842.62,673.60,636.84,782.38,1015.97
71,1015.97,0,0,2025-06-09,881.72,802.16,756.42,810.97,546.93,571.22,708.74,1014.76


In [3]:
from data_split import train_test_split
from datetime import datetime

# Realiza el split usando la fecha de corte adecuada y el target
X_train, y_train, X_test, y_test = train_test_split(
    df,
    split_date=datetime(2025, 3, 3),  # Ajusta la fecha si es necesario
    target='base_imponible'
)

In [ ]:
# SARIMA: Entrenamiento y predicción con statsmodels
import statsmodels.api as sm
import numpy as np

# Usamos solo la variable target y el índice temporal
y_train_series = y_train.copy()
y_train_series.index = X_train['week_start'] if 'week_start' in X_train.columns else X_train.index
y_test_series = y_test.copy()
y_test_series.index = X_test['week_start'] if 'week_start' in X_test.columns else X_test.index

# Parámetros SARIMA: (p,d,q) x (P,D,Q,s)
# p,d,q: orden ARIMA; P,D,Q: orden estacional; s: periodo estacional (ej: 52 para semanas en año)
sarima_order = (1,1,1)
seasonal_order = (1,1,1,52)  # Ajusta '52' según tu frecuencia estacional

# Entrenamiento SARIMA
sarima_model = sm.tsa.statespace.SARIMAX(y_train_series, order=sarima_order, seasonal_order=seasonal_order)
sarima_results = sarima_model.fit(disp=False)

# Predicción en test
y_pred_sarima = sarima_results.predict(start=y_test_series.index[0], end=y_test_series.index[-1])

# Evaluación
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test_series, y_pred_sarima)
mse = mean_squared_error(y_test_series, y_pred_sarima)
print(f"MAE SARIMA: {mae:.2f}, MSE SARIMA: {mse:.2f}")

In [6]:
from sklearn.metrics import mean_absolute_error,r2_score, root_mean_squared_error , mean_absolute_percentage_error, r2_score

# Realizamos las predicciones
y_pred_lr = lr_model.predict(X_test)

# Evaluamos el modelo
mae = mean_absolute_error(y_test, y_pred_lr)
rmse = root_mean_squared_error(y_test, y_pred_lr)
mape = mean_absolute_percentage_error(y_test, y_pred_lr)
r2 = r2_score(y_test, y_pred_lr)

print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2F} ,R2: {r2:.2f}")

MAE: 90.98, RMSE: 118.46, MAPE: 0.12 ,R2: 0.72


In [7]:
import joblib
from paths import MODELS_DIR
import os

# Guardamos el modelo
joblib.dump(lr_model, MODELS_DIR / 'linear_regression.pkl')

['C:\\Workspace\\mlops_fleca_project\\models\\linear_regression.pkl']